In [1]:
# Дан файл с некоторыми данными. Формат файла – произвольный. 
# Спроектируйте на его основе и создайте таблицу в базе данных (SQLite). 
# Считайте данные из файла и запишите их в созданную таблицу. 
# Реализуйте и выполните следующие запросы:
#    вывод первых (VAR+10) отсортированных по произвольному числовому полю строк из таблицы в файл формата json;
#    вывод (сумму, мин, макс, среднее) по произвольному числовому полю;
#    вывод частоты встречаемости для категориального поля;
#    вывод первых (VAR+10) отфильтрованных по произвольному предикату отсортированных по произвольному числовому полю строк 
#    из таблицы в файл формате json.

In [12]:
import pandas as pd
import sqlite3
import msgpack
import json

In [13]:
with open(b'task_1_var_63_item.msgpack', 'rb') as f:
    data = msgpack.load(f)
data[0:3]    

[{'id': 305384,
  'name': 'Балтийская цепь 1986',
  'city': 'Мадрид',
  'begin': '16.03',
  'system': 'circular',
  'tours_count': 9,
  'min_rating': 2227,
  'time_on_game': 42},
 {'id': 990466,
  'name': 'Цюрих 2024',
  'city': 'Осера',
  'begin': '21.05',
  'system': 'Olympic',
  'tours_count': 11,
  'min_rating': 2377,
  'time_on_game': 58},
 {'id': 263581,
  'name': 'Реджо-нель-Эмилия 1959',
  'city': 'Краков',
  'begin': '07.10',
  'system': 'Olympic',
  'tours_count': 8,
  'min_rating': 2237,
  'time_on_game': 78}]

In [14]:
def connect_to_db(file):
    connection = sqlite3.connect(file)
    connection.row_factory = sqlite3.Row
    return connection

In [15]:
# Создание таблицы в базе данных
def create_table():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS properties
                  (id INT PRIMARY KEY,
                  name TEXT,
                  city TEXT,
                  begin TXT,
                  system TXT,
                  tours_count INT,
                  min_rating INT,
                  time_on_game INT)''')
    conn.commit()
    conn.close()

In [23]:
# Чтение данных из файла в таблицу
def insert_data():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    # Чтение данных из файла task_1_var_63_item.msgpack
    with open('task_1_var_63_item.msgpack', 'rb') as file:
        data = msgpack.load(file)
    
    rows = []
    for item in data:
        row = {
            'id': item['id'],
            'name': item['name'], 
            'city': item['city'], 
            'begin': item['begin'], 
            'system': item['system'], 
            'tours_count': item['tours_count'], 
            'min_rating': item['min_rating'], 
            'time_on_game': item['time_on_game']
        }
        rows.append(row)
        
    # Запись данных в таблицу
    c.executemany("INSERT OR IGNORE INTO properties (id, name, city, begin, system, tours_count, min_rating, time_on_game) VALUES (:id, :name, :city, :begin, :system, :tours_count, :min_rating, :time_on_game)", rows)
    # Сохранение изменений
    conn.commit()
    conn.close()    

In [32]:
# Вывод первых 73 отсортированных по числовому полю ['tours_count'] в файл json
def sort_by_tours_count():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute("SELECT * FROM properties ORDER BY tours_count LIMIT 73")
    result = c.fetchall()
    with open('sorted_properties_63_1.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()

In [41]:
# Вывод суммы, минимума, максимума и среднего значения по полю ['min_rating']
def min_rating_statistics():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT SUM(min_rating), MIN(min_rating), MAX(min_rating), AVG(min_rating) FROM properties")
    result = c.fetchone()

    conn.close()

    return result

In [42]:
# Вывод частоты встречаемости для категориального поля ['city']
def city_frequency():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT city, COUNT(*) FROM properties GROUP BY city")
    result = c.fetchall()

    conn.close()

    return result

In [51]:
# Вывод первых 73 отфильтрованных по произвольному предикату и отсортированных по числовому полю ['time_on_game'] в файл json
def filter_and_sort_by_time_on_game():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT * FROM properties WHERE time_on_game > 80 ORDER BY time_on_game LIMIT 73")
    result = c.fetchall()

    with open('filtered_properties_63_1.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()

In [52]:
connect_to_db('database.db')

In [53]:
# Создание таблицы и запись данных
create_table()
insert_data()

In [55]:
# Вывод первых 73 отсортированных по числовому полю ['tours_count'] в файл json
sort_by_tours_count()

In [56]:
# Вывод статистики по полю ['min_rating']
min_rating_stats = min_rating_statistics()
min_rating_sum, min_rating_min, min_rating_max, min_rating_avg = min_rating_stats
min_rating_stats

(174401, 2217, 2641, 2422.2361111111113)

In [57]:
# Вывод частоты встречаемости для категориального поля ['city']
city_freq = city_frequency()
city_freq

[('Авилес', 1),
 ('Алькала-де-Энарес', 1),
 ('Астана', 1),
 ('Баку', 3),
 ('Барселона', 1),
 ('Бланес', 1),
 ('Будапешт', 2),
 ('Вальядолид', 2),
 ('Виго', 2),
 ('Вильнюс', 1),
 ('Вильявисиоса', 2),
 ('Вроцлав', 1),
 ('Душанбе', 2),
 ('Картахена', 2),
 ('Касерес', 4),
 ('Киев', 3),
 ('Кишинев', 1),
 ('Кордова', 1),
 ('Краков', 2),
 ('Куэнка', 1),
 ('Лас-Росас', 1),
 ('Луарка', 1),
 ('Луго', 1),
 ('Мадрид', 2),
 ('Малага', 1),
 ('Мерида', 1),
 ('Минск', 1),
 ('Монсон', 1),
 ('Москва', 3),
 ('Мурсия', 1),
 ('Овьедо', 1),
 ('Осера', 3),
 ('Пласенсия', 1),
 ('Рига', 1),
 ('Сан-Себастьян', 1),
 ('Санкт-Петербург', 2),
 ('Сантьяго-де-Компостела', 1),
 ('Санхенхо', 1),
 ('Сеговия', 1),
 ('София', 2),
 ('Сьюдад-Реаль', 1),
 ('Таллин', 2),
 ('Тарраса', 2),
 ('Ташкент', 1),
 ('Трухильо', 1),
 ('Хихон', 2),
 ('Эль-Пуэрто-де-Санта-Мария', 1),
 ('Эльче', 1)]

In [15]:
# Вывод первых 73 отфильтрованных по произвольному предикату и отсортированных по числовому полю ['time_on_game'] в файл json
filter_and_sort_by_time_on_game()